In [3]:
import math
from collections import Counter

In [ ]:
"""
When to Use Decision Trees
Use decision trees when:

You need interpretable models that stakeholders can easily understand
Working with mixed data types (categorical and numerical features)
Feature interactions are important in your problem
You don't have time for extensive feature preprocessing
The relationship between features and target is non-linear
You're doing exploratory data analysis to understand feature importance
Working with tabular data problems
You need a baseline model quickly
"""

In [6]:
import math
from collections import Counter

class DecisionTree:
    def __init__(self, max_depth=5, min_samples_split=2):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.root = None

    def entropy(self, y):
        if len(y) == 0:
            return 0
        counts = Counter(y)
        total = len(y)
        return -sum((count / total) * math.log2(count / total) for count in counts.values() if count > 0)

    def information_gain(self, X, y, feature_idx, threshold):
        left = [i for i in range(len(X)) if X[i][feature_idx] <= threshold]
        right = [i for i in range(len(X)) if X[i][feature_idx] > threshold]
        if not left or not right:
            return 0

        left_y = [y[i] for i in left]
        right_y = [y[i] for i in right]
        total = len(y)
        gain = self.entropy(y)
        gain -= (len(left_y) / total) * self.entropy(left_y)
        gain -= (len(right_y) / total) * self.entropy(right_y)
        return gain

    def find_best_split(self, X, y):
        best_gain = 0
        best_feature = None
        best_threshold = None
        n_features = len(X[0])

        for feature_idx in range(n_features):
            values = sorted(set(row[feature_idx] for row in X))
            for threshold in values:
                gain = self.information_gain(X, y, feature_idx, threshold)
                if gain > best_gain:
                    best_gain = gain
                    best_feature = feature_idx
                    best_threshold = threshold

        return best_feature, best_threshold, best_gain

    def build_tree(self, X, y, depth=0):
        if depth >= self.max_depth or len(y) < self.min_samples_split or len(set(y)) == 1:
            return Counter(y).most_common(1)[0][0]

        feature, threshold, gain = self.find_best_split(X, y)
        if gain == 0:
            return Counter(y).most_common(1)[0][0]

        left_idx = [i for i in range(len(X)) if X[i][feature] <= threshold]
        right_idx = [i for i in range(len(X)) if X[i][feature] > threshold]

        left_X = [X[i] for i in left_idx]
        left_y = [y[i] for i in left_idx]
        right_X = [X[i] for i in right_idx]
        right_y = [y[i] for i in right_idx]

        return {
            'feature': feature,
            'threshold': threshold,
            'left': self.build_tree(left_X, left_y, depth + 1),
            'right': self.build_tree(right_X, right_y, depth + 1)
        }

    def fit(self, X, y):
        self.root = self.build_tree(X, y)

    def predict_single(self, x, node):
        if not isinstance(node, dict):
            return node
        if x[node['feature']] <= node['threshold']:
            return self.predict_single(x, node['left'])
        return self.predict_single(x, node['right'])

    def predict(self, X):
        return [self.predict_single(x, self.root) for x in X]

    def print_tree(self, node=None, depth=0, prefix="Root"):
        if node is None:
            node = self.root
        indent = "  " * depth
        if isinstance(node, dict):
            print(f"{indent}{prefix}: Feature {node['feature']} <= {node['threshold']}")
            self.print_tree(node['left'], depth + 1, "Left")
            self.print_tree(node['right'], depth + 1, "Right")
        else:
            print(f"{indent}{prefix}: Predict '{node}'")

In [7]:

if __name__ == "__main__":
    X_train = [
        [85, 85, 0], [80, 90, 1], [83, 78, 0], [70, 96, 0],
        [68, 80, 0], [65, 70, 1], [64, 65, 1], [72, 95, 0],
        [69, 70, 0], [75, 80, 0], [75, 70, 1], [72, 90, 1],
        [81, 75, 0], [71, 91, 1]
    ]

    y_train = [
        'no_play', 'no_play', 'play', 'play',
        'play', 'no_play', 'play', 'no_play',
        'play', 'play', 'no_play', 'play',
        'play', 'play'
    ]

    dt = DecisionTree(max_depth=3)
    dt.fit(X_train, y_train)

    print("Decision Tree Structure:")
    print("=" * 40)
    dt.print_tree()

    X_test = [
        [70, 75, 0],
        [85, 90, 1],
        [65, 70, 0],
    ]

    predictions = dt.predict(X_test)
    print("\nPredictions:")
    print("=" * 40)
    for i, (features, pred) in enumerate(zip(X_test, predictions), 1):
        print(f"Test {i}: {features} -> {pred}")

    train_predictions = dt.predict(X_train)
    accuracy = sum(1 for true, pred in zip(y_train, train_predictions) if true == pred) / len(y_train)
    print(f"\nTraining Accuracy: {accuracy:.2%}")


Decision Tree Structure:
Root: Feature 0 <= 83
  Left: Feature 2 <= 0
    Left: Feature 1 <= 80
      Left: Predict 'play'
      Right: Predict 'play'
    Right: Feature 0 <= 72
      Left: Predict 'play'
      Right: Predict 'no_play'
  Right: Predict 'no_play'

Predictions:
Test 1: [70, 75, 0] -> play
Test 2: [85, 90, 1] -> no_play
Test 3: [65, 70, 0] -> play

Training Accuracy: 85.71%


In [ ]:
#Stop splitting whenever you get zero in the leaf node 
#whenever the entropy is 0 that is the pure node
#info gain tells which split is better out of all and which one should be used in the decision tree
#the higher the info gain is better the split
#theres more on dt